====================================================================
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
### 使用FFmpeg將音頻MP4轉換為MP3?之討論

#### 透過chatGPT -1
##### Question :使用FFmpeg將音頻MP4轉換為MP3?
##### Answer:
要處理在執行下載過程中遇到長時間無法完成的情況，可以使用 signal 模組來設定超時處理。在超時發生時，會拋出一個自定義的例外，從而中斷下載過程並打印錯誤訊息。以下是示範程式碼：
要使用 FFmpeg 將音頻從 MP4 轉換為 MP3，您可以使用以下命令：

>>>ffmpeg -i input.mp4 -vn -acodec libmp3lame -q:a 2 output.mp3

這裡是這個命令的詳細說明：
1. ffmpeg: 調用 FFmpeg 工具。
2. -i input.mp4: 指定輸入文件 input.mp4。
3. -vn: 禁用視頻，因為我們只需要音頻部分。
4. -acodec libmp3lame: 指定 MP3 編碼器。libmp3lame 是一個高質量的 MP3 編碼器。
5. -q:a 2: 設置音頻質量。q:a 選項允許設置 VBR（可變比特率）的質量級別，數字 2 表示非常高的質量（範圍從 0 到 9，0 為最佳）。
6. output.mp3: 指定輸出文件 output.mp3。

您可以將此命令用於命令行中，也可以在 Python 程序中通過 subprocess 模組調用它。這裡有一個示例 Python 程序，演示如何使用 subprocess 來運行 FFmpeg 命令：

In [ ]:
import subprocess

def convert_mp4_to_mp3(input_file, output_file):
    command = [
        'ffmpeg',
        '-i', input_file,
        '-vn',
        '-acodec', 'libmp3lame',
        '-q:a', '2',
        output_file
    ]
    
    try:
        result = subprocess.run(command, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
        print(f"Conversion successful! MP3 file is located at: {output_file}")
    except subprocess.CalledProcessError as e:
        print("An error occurred during conversion:", e)
        if e.stderr:
            print("FFmpeg error output:", e.stderr)
        else:
            print("Could not retrieve FFmpeg error output.")

# Example usage
convert_mp4_to_mp3("input.mp4", "output.mp3")


這個程序調用 convert_mp4_to_mp3 函數，將 input.mp4 轉換為 output.mp3。如果轉換成功，它會打印成功消息；如果發生錯誤，則會打印錯誤消息和 FFmpeg 的錯誤輸出。

#### 透過chatGPT -2
##### Question : 當abr=128kbps 時，-q:a 要設置為何?
##### Answer:
在FFmpeg中，-q:a 和 -b:a 是用來設定音頻質量和比特率的參數。這兩者通常不會同時使用。-q:a（質量）用來控制VBR（可變比特率）的音質，數值範圍從0到9，數值越小音質越好。而 -b:a 用來設定CBR（固定比特率）的音頻比特率。

如果你想要設定音頻比特率為128kbps，應該使用 -b:a 而不是 -q:a。

以下是使用 -b:a 的正確方式

In [ ]:
def convert_mp4_to_mp3(input_file, output_file):
    command = [
        'ffmpeg',
        '-i', input_file,
        '-vn',
        '-acodec', 'libmp3lame',
        '-b:a', '128k',
        output_file
    ]
    try:
        result = subprocess.run(command, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
        print("轉換成功！輸出文件位於:", output_file)
    except subprocess.CalledProcessError as e:
        print("ffmpeg轉換過程中發生錯誤:", e)
        if e.stderr:
            print("ffmpeg命令輸出:", e.stderr)
        else:
            print("無法獲取 ffmpeg 錯誤輸出。")

# 範例使用
convert_mp4_to_mp3("input.mp4", "output.mp3")


這樣可以確保音頻的比特率是128kbps。如果你還是想要使用 -q:a 來設定音質的話，你需要注意，-q:a 和比特率之間沒有一個簡單的直接對應，通常需要實驗和經驗來找出適合你的設置。對應 -q:a 設置的比特率值大致如下：

-q:a 0 大約是 320 kbps \n
-q:a 1 大約是 256 kbps \n
-q:a 2 大約是 192 kbps \n
-q:a 3 大約是 160 kbps \n
-q:a 4 大約是 128 kbps \n
-q:a 5 大約是 96 kbps \n
所以如果你希望音質接近128kbps，可以設定 -q:a 4。但請注意這只是近似值。

In [ ]:
def convert_mp4_to_mp3(input_file, output_file):
    command = [
        'ffmpeg',
        '-i', input_file,
        '-vn',
        '-acodec', 'libmp3lame',
        '-q:a', '4',
        output_file
    ]
    try:
        result = subprocess.run(command, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
        print("轉換成功！輸出文件位於:", output_file)
    except subprocess.CalledProcessError as e:
        print("ffmpeg轉換過程中發生錯誤:", e)
        if e.stderr:
            print("ffmpeg命令輸出:", e.stderr)
        else:
            print("無法獲取 ffmpeg 錯誤輸出。")

# 範例使用
convert_mp4_to_mp3("input.mp4", "output.mp3")


       ~~~ End ~~~
+++++++++++++++++++++++++++++
=============================

In [7]:
# V0.80 驗證0
# get_audio_only V0.6 --> 2024.06.19 final
# 定義全局變量 global
# 設定包含中英文字

from pytube import YouTube
import re

# 定義全局變量
global_yt_title = None

def get_audio_only(url):
    yt = YouTube(url)   
    print('全名 : ',yt.title)
    list=['/',':','"','<','>','\?','\*']
    yt_titles = yt.title
    for i in list:
        yt_title = re.sub(f'{i}','_',yt_titles)
        yt_titles = yt_title
    print(yt_titles)

    yt_title = re.search(r'[A-Za-z0-9_ \u4e00-\u9fff\s]+',yt_titles).group() #設定包含中英文字
    print('簡稱 : ',yt_title)
    global global_yt_title   # 定義全局變量
    global_yt_title = yt_title    # 定義全局變量

    yt_audio = yt.streams.filter(abr='128kbps').first()
    yt_audio.download(output_path='C:/Users/Lu/Documents/youtube',filename=f'yt_audio_{yt_title}.mp4', skip_existing=True, max_retries=0)   #執行ok
    print(f'downdload < {yt.title} > 完成')
    return yt_title

def convert_mp4_to_mp3(input_file, output_file):
    command = [
        'ffmpeg',
        '-i', input_file,
        '-vn',
        '-acodec', 'libmp3lame',
        '-q:a', '3',
        output_file
    ]
    # -q:a 3 大約是 160 kbps
    try:
        result = subprocess.run(command, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True, encoding='utf-8')   #追加, encoding='utf-8'
        print("轉換成功！輸出文件位於:", output_file)
    except subprocess.CalledProcessError as e:
        print("ffmpeg轉換過程中發生錯誤:", e)
        if e.stderr:
            print("ffmpeg命令輸出:", e.stderr)
        else:
            print("無法獲取 ffmpeg 錯誤輸出。")


url = 'https://www.youtube.com/watch?v=b5d5OmmUlPc'  # 曲目:2023宮崎駿鋼琴音樂合集-最佳放鬆音樂《100%無廣告》スタジオジブリGhibli Relaxing Music【スターバックス】工作-讀書專用音樂
get_audio_only(url)

yt_title = global_yt_title     # 定義全局變量

convert_mp4_to_mp3(f'C:/Users/Lu/Documents/youtube/yt_audio_{yt_title}.mp4', f'C:/Users/Lu/Documents/youtube/yt_audio_{yt_title}.mp3')

# media 長度 01:33:15 mp4容量大小88,433KB mp3容量大小87,074KB 處理耗時2m 2.9s

全名 :  2023宮崎駿鋼琴音樂合集-最佳放鬆音樂《100%無廣告》スタジオジブリGhibli Relaxing Music【スターバックス】工作-讀書專用音樂
2023宮崎駿鋼琴音樂合集-最佳放鬆音樂《100%無廣告》スタジオジブリGhibli Relaxing Music【スターバックス】工作-讀書專用音樂
簡稱 :  2023宮崎駿鋼琴音樂合集
downdload < 2023宮崎駿鋼琴音樂合集-最佳放鬆音樂《100%無廣告》スタジオジブリGhibli Relaxing Music【スターバックス】工作-讀書專用音樂 > 完成
轉換成功！輸出文件位於: C:/Users/Lu/Documents/youtube/yt_audio_2023宮崎駿鋼琴音樂合集.mp3
